# LLMのRAGを使ったQABOT

1. **データ取得**:
    - ICML 2024の論文PDFを取得し、./icml_2024_papersディレクトリに格納する    <---- now
2. **データの埋め込みベクトル化**:
    - 取得したPDFからテキストを抽出し、ベクトル化する。
3. **LLMのRAGモデルの設定**:
    - 質問に対して関連する文書を検索し、その文書を基に回答を生成するRAGモデルを設定する。
4. **QABOTの構築**:
    - ユーザーインターフェースを作成し、ユーザーからの質問に対してRAGモデルを使って回答するQABOTを構築する。



---

<!-- 旧 -->
<!-- 1. **データ取得**:
    - ICML 2024の論文PDFを取得し、./icml_2024_papersディレクトリに格納する    <---- now
2. **データの埋め込みベクトル化**:
    - 取得したPDFからテキストを抽出し、ベクトル化する。
3. **検索インデックスの構築**:
    - ベクトル化されたデータを元に検索インデックスを構築する。
4. **LLMのRAGモデルの設定**:
    - 質問に対して関連する文書を検索し、その文書を基に回答を生成するRAGモデルを設定する。
5. **QABOTの構築**:
    - ユーザーインターフェースを作成し、ユーザーからの質問に対してRAGモデルを使って回答するQABOTを構築する。 -->

In [1]:
!pip3 install selenium
!pip3 install webdriver-manager
!pip3 install arxiv==2.1.0
!pip3 install tqdm

In [2]:
import os
import time
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
import arxiv

##  ICMLサイトから論文タイトルを取得

In [3]:

# ICML 2024の採用論文リストページのURL
icml_url = "https://icml.cc/Downloads/2024?csrfmiddlewaretoken=OtP8bIlzu8ISV2jyQxdmHJD51dqmpzXSrpBgjNZnTgJ4w8ulxBeo23ApLLswZyhg&file_format=&posters=on&submitaction=Update+List&submitaction=Update+List"

# ページを取得
response = requests.get(icml_url)
soup = BeautifulSoup(response.content, 'html.parser')

# 論文タイトルを抽出
titles = []
for link in soup.find_all('a', class_='Poster'):
    titles.append(link.text.strip())

# タイトルを確認
# for title in titles:
#     print(title)

In [4]:
len(titles)

2634

## Arxiv で論文を検索してChatGPT or LLMが概要に入っているPDFをダウンロード

In [5]:
# 保存するディレクトリ
save_dir = "./icml_2024_papers"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [6]:
def download_pdf_from_arxiv(title, save_dir):
    search = arxiv.Search(
        query=title,
        max_results=1,
        sort_by=arxiv.SortCriterion.Relevance
    )
    for result in search.results():
        if 'ChatGPT' in result.summary or 'LLM' in result.summary or 'large language model' in result.summary:
            pdf_url = result.pdf_url
            response = requests.get(pdf_url, stream=True)
            with open(os.path.join(save_dir, f"{title}.pdf"), 'wb') as f:
                for data in response.iter_content(1024):
                    f.write(data)
            # print(f"Downloaded: {title}")
        # else:
            # print(f"Skipped: {title} (no mention of ChatGPT or LLM)")

In [7]:
# タイトルを使ってArxivからPDFをダウンロード
for title in tqdm(titles, desc="Total Progress"):
    try:
        download_pdf_from_arxiv(title, save_dir)
        time.sleep(1)  # ダウンロード間隔を1秒に設定
    except Exception as e:
        print(f"Failed to download {title}: {e}")

Total Progress:   0%|          | 0/2634 [00:00<?, ?it/s]/tmp/ipykernel_1797/4033538657.py:7: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():
Total Progress: 100%|██████████| 2634/2634 [2:42:45<00:00,  3.71s/it]   
